In [1]:
!pip install tensorflow

  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached wrapt-1.17.2-cp312-cp312-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached h5py-3.13.0-cp312-cp312-win_amd64.whl.metadata (2.5 kB)
  Using cached ml_dtypes-0.5.1-cp312-cp312-win_amd64.whl.metadata (22 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached optree-0.15.0-cp312-cp312-win_amd64.whl.metadata (49 kB)
  Using cached markdown-3.8-p


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install tensorflow_hub

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from PIL import Image, ImageDraw, ImageFont, ImageOps
import requests
from io import BytesIO

In [ ]:
# Load the model from TF Hub
detector = hub.load("https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1").signatures['default']

# Classes you care about
TARGET_CLASSES = set(["Food processor", "Fast food", "Food", "Seafood", "Snack"])


In [ ]:
def load_image_from_url(url, size=(640, 480)):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content)).convert("RGB")
    img = ImageOps.fit(img, size, Image.Resampling.LANCZOS)
    return img


In [31]:

def run_object_detection(image: Image.Image):
    image_np = np.array(image)
    # Convert to tensor without specifying dtype
    input_tensor = tf.convert_to_tensor(image_np)[tf.newaxis, ...]
    # Convert to float32 and normalize to [0,1]
    input_tensor = tf.cast(input_tensor, tf.float32) / 255.0
    results = detector(input_tensor)
    results = {k: v.numpy() for k, v in results.items()}
    return results, image_np

def get_filtered_class_boxes(results):
    # for same class, keep the one with the highest score
    # and remove duplicates
    boxes = []
    classes = []
    scores = []
                    
    for i in range(len(results["detection_scores"])):
        class_name = results["detection_class_entities"][i].decode("utf-8")
        box = results["detection_boxes"][i]
        score = results["detection_scores"][i]
        if class_name in TARGET_CLASSES:
            if class_name not in classes:
                boxes.append(box)
                classes.append(class_name)
                scores.append(score)
            else:
                index = classes.index(class_name)
                if score > scores[index]:
                    boxes[index] = box
                    classes[index] = class_name
                    scores[index] = score
    return boxes, classes, scores


In [35]:

def crop_and_save(image_np, boxes, class_names, scores, min_score=0.3):
    cropped_images = []
    for i in range(len(scores)):
        if scores[i] > min_score:
            ymin, xmin, ymax, xmax = boxes[i]
            im_width, im_height = image_np.shape[1], image_np.shape[0]
            (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                          ymin * im_height, ymax * im_height)
            cropped_image = image_np[int(top):int(bottom), int(left):int(right)]
            cropped_images.append((cropped_image, class_names[i], scores[i]))
            # Save the cropped image
            pil_image = Image.fromarray(cropped_image)
            pil_image.save(f"output/{class_names[i]}_{scores[i]:.2f}.jpg")
    return cropped_images

def draw_boxes(image_np, boxes, class_names, scores, min_score=0.3):
    image_pil = Image.fromarray(image_np)
    draw = ImageDraw.Draw(image_pil)
    font = ImageFont.load_default()

    for i in range(len(scores)):
        label = class_names[i]
        print(label, scores[i])
        if label in TARGET_CLASSES and scores[i] > min_score:
            ymin, xmin, ymax, xmax = boxes[i]
            im_width, im_height = image_pil.size
            (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                          ymin * im_height, ymax * im_height)
            draw.rectangle([left, top, right, bottom], outline="red", width=2)
            draw.text((left, top), f"{label}: {scores[i]*100:.1f}%", fill="white", font=font)
    return image_pil

In [21]:
def detect_and_display(image):
    results, image_np = run_object_detection(image)
    
    final_image = draw_boxes(image_np,
                             results["detection_boxes"],
                             results["detection_class_entities"],
                             results["detection_scores"])
    final_image.show()

In [24]:
def detect_and_save(image):
    results, image_np = run_object_detection(image)
    boxes, class_names, scores = get_filtered_class_boxes(results)
    cropped_images = crop_and_save(image_np, boxes, class_names, scores,0)
    return cropped_images

In [ ]:
detect_and_display(
    image = Image.open("bv.jpg").convert("RGB"))

In [36]:
detect_and_save(
    image = Image.open("bv.jpg").convert("RGB"))

[(array([[[122, 113, 108],
          [123, 114, 109],
          [123, 114, 109],
          ...,
          [150, 141, 136],
          [150, 141, 136],
          [151, 142, 137]],
  
         [[122, 113, 108],
          [123, 114, 109],
          [123, 114, 109],
          ...,
          [150, 141, 136],
          [150, 141, 136],
          [151, 142, 137]],
  
         [[122, 113, 108],
          [123, 114, 109],
          [123, 114, 109],
          ...,
          [150, 141, 136],
          [150, 141, 136],
          [151, 142, 137]],
  
         ...,
  
         [[ 68,  31,  25],
          [ 69,  31,  28],
          [ 66,  31,  27],
          ...,
          [128, 119, 114],
          [128, 119, 114],
          [126, 117, 112]],
  
         [[ 64,  29,  23],
          [ 64,  29,  25],
          [ 62,  28,  26],
          ...,
          [127, 120, 114],
          [126, 119, 113],
          [126, 119, 113]],
  
         [[ 62,  29,  24],
          [ 58,  27,  24],
          [ 58,  27,  24